In [19]:
import sys
sys.path.insert(1, "/home/p11333at/nlp-project/script/")
from tweets_collector import *
import re
import os
import json

### Collect tweets count for recent 7 days

In [20]:
collector = TweetsCollector()

query = "(covid OR covid19 OR covid-19 OR coronavirus OR (corona virus) OR pandemic) -is:retweet lang:en"

day_counts = collector.client.get_recent_tweets_count(query=query, granularity='day')


In [21]:
output = {}
for i, day in enumerate(day_counts.data):
    if i == 0 or i == len(day_counts.data)-1:
        continue
    date = re.search(r"\d{4}-\d{2}-\d{2}", day["start"]).group(0)
    count = day["tweet_count"]
    output.update(
        {date : 
            {"total" : count,
             "hourly" : {}}
        }
    )
output

{'2023-02-27': {'total': 126965, 'hourly': {}},
 '2023-02-28': {'total': 141997, 'hourly': {}},
 '2023-03-01': {'total': 136187, 'hourly': {}},
 '2023-03-02': {'total': 120969, 'hourly': {}},
 '2023-03-03': {'total': 109937, 'hourly': {}},
 '2023-03-04': {'total': 95729, 'hourly': {}}}

In [22]:
# backward_days = 1   # Set to 1 to collect yesterday's (Max: 6) 
# collection_date = today(backward_days)
# start_time = collection_date
# end_time = today(backward_days) + timedelta(days=1, seconds=-1)
# print(f"The date: '{collection_date.strftime('%Y-%m-%dT%H:%M:%SZ')}' -> \033[32m{backward_days} day(s) ago\033[0m")

start_time = today(6)
end_time = today(1) + timedelta(days=1)
print(f"The start time : '{start_time.strftime('%Y-%m-%dT%H:%M:%SZ')}'")
print(f"The  end  time : '{end_time.strftime('%Y-%m-%dT%H:%M:%SZ')}'")

hour_counts = collector.client.get_recent_tweets_count(query=query, granularity='hour', start_time=start_time, end_time=end_time)


The start time : '2023-02-27T00:00:00Z'
The  end  time : '2023-03-05T00:00:00Z'


In [5]:
for hour in hour_counts.data:
    start = hour["start"]
    date = re.search(r"\d{4}-\d{2}-\d{2}", start).group(0)
    time = re.search(r"\d{2}:\d{2}:\d{2}", start).group(0)
    # print(time, ":", hour["tweet_count"])
    output[date]["hourly"].update({time : hour["tweet_count"]})

# start, hour_counts.data

### Store results in dict

### Output results to file

In [6]:
filename_meta = "meta.json"
path = "/home/p11333at/nlp-project/data/"

if os.path.exists(f"{path}{filename_meta}"):
    with open(f"{path}{filename_meta}", "r+") as f:
        meta = json.load(f)
        meta.update(output)

        f.seek(0)
        json.dump(meta, f, indent=2)
        f.truncate()
else:
    with open(f"{path}{filename_meta}", "w") as f:
        json.dump(output, f, indent=2)

print(f"The tweet counts is recorded in '{filename_meta}' file.")

The tweet counts is recorded in 'meta.json' file.


END